In [1]:
!pip install sodapy
!pip install feather-format

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;
import re;
from time import time
import math
import feather
from collections import defaultdict

In [3]:
# Set up Socrata
client = Socrata("data.sfgov.org", None)
socrata_id_older = "cuks-n6tp"
socrata_id_newer = "wg3w-h783"

In [4]:
# Get Metadata for 2003-2018
metadata_old = client.get_metadata(socrata_id_older)
[x['name'] for x in metadata_old['columns']]

[u'IncidntNum',
 u'Category',
 u'Descript',
 u'DayOfWeek',
 u'Date',
 u'Time',
 u'PdDistrict',
 u'Resolution',
 u'Address',
 u'X',
 u'Y',
 u'Location',
 u'Location (address)',
 u'Location (city)',
 u'Location (state)',
 u'Location (zip)',
 u'PdId',
 u'SF Find Neighborhoods',
 u'Current Police Districts',
 u'Current Supervisor Districts',
 u'Analysis Neighborhoods',
 u':@computed_region_yftq_j783',
 u':@computed_region_p5aj_wyqh',
 u':@computed_region_rxqg_mtj9',
 u':@computed_region_bh8s_q3mv',
 u':@computed_region_fyvs_ahh9',
 u':@computed_region_9dfj_4gjx',
 u':@computed_region_n4xg_c4py',
 u':@computed_region_4isq_27mq',
 u':@computed_region_fcz8_est8',
 u':@computed_region_pigm_ib2e',
 u':@computed_region_9jxd_iqea',
 u':@computed_region_6pnf_4xz7',
 u':@computed_region_6ezc_tdp2',
 u':@computed_region_h4ep_8xdi',
 u':@computed_region_nqbw_i6c3',
 u':@computed_region_2dwj_jsy4']

In [5]:
# Get Metadata for 2018-2019
metadata_new = client.get_metadata(socrata_id_newer)
[x['name'] for x in metadata_new['columns']]

[u'Incident Datetime',
 u'Incident Date',
 u'Incident Time',
 u'Incident Year',
 u'Incident Day of Week',
 u'Report Datetime',
 u'Row ID',
 u'Incident ID',
 u'Incident Number',
 u'CAD Number',
 u'Report Type Code',
 u'Report Type Description',
 u'Filed Online',
 u'Incident Code',
 u'Incident Category',
 u'Incident Subcategory',
 u'Incident Description',
 u'Resolution',
 u'Intersection',
 u'CNN',
 u'Police District',
 u'Analysis Neighborhood',
 u'Supervisor District',
 u'Latitude',
 u'Longitude',
 u'point',
 u'SF Find Neighborhoods',
 u'Current Police Districts',
 u'Current Supervisor Districts',
 u'Analysis Neighborhoods',
 u'HSOC Zones as of 2018-06-05',
 u'OWED Public Spaces',
 u'Central Market/Tenderloin Boundary Polygon - Updated',
 u'Parks Alliance CPSI (27+TL sites)']

In [6]:
col_new = []
col_old = []

for i in range(len(metadata_new['columns'])):
    name = metadata_new['columns'][i]['name']
    names = []
    names.append(name)
    col_new = col_new + names

for i in range(len(metadata_old['columns'])):
    name = metadata_old['columns'][i]['name']
    names = []
    names.append(name)
    col_old = col_old + names

In [7]:
# Import 2018-2019 data via Socrata

loop_size = 10000
num_loops = 20

df_new = pd.DataFrame(columns=col_new)

for i in range(num_loops):
    results = client.get(socrata_id_newer,
                         limit=loop_size,
                         offset=loop_size * i)
    print("\n> Loop number: {}".format(i))
    df_loop = pd.DataFrame(results)
    df_new = pd.concat([df_new,df_loop])


> Loop number: 0

> Loop number: 1

> Loop number: 2

> Loop number: 3

> Loop number: 4

> Loop number: 5

> Loop number: 6

> Loop number: 7

> Loop number: 8

> Loop number: 9

> Loop number: 10

> Loop number: 11

> Loop number: 12

> Loop number: 13

> Loop number: 14

> Loop number: 15

> Loop number: 16

> Loop number: 17

> Loop number: 18

> Loop number: 19


In [8]:
# Import 2003-2018 data via Socrata 
loop_size = 25000
num_loops = 21

df_old = pd.DataFrame(columns=col_old)

for i in range(num_loops):
    results = client.get(socrata_id_older,
                         where="date_extract_y(Date) > 2014",
                         limit=loop_size,
                         offset=loop_size * i)
    print("\n> Loop number: {}".format(i))
    df_loop = pd.DataFrame(results)
    df_old = pd.concat([df_old,df_loop])


> Loop number: 0

> Loop number: 1

> Loop number: 2

> Loop number: 3

> Loop number: 4

> Loop number: 5

> Loop number: 6

> Loop number: 7

> Loop number: 8

> Loop number: 9

> Loop number: 10

> Loop number: 11

> Loop number: 12

> Loop number: 13

> Loop number: 14

> Loop number: 15

> Loop number: 16

> Loop number: 17

> Loop number: 18

> Loop number: 19

> Loop number: 20


In [9]:
print("Data - 2018 to 2019")
print(df_new.shape)
print(df_new.columns)

print("Data - 2014 to 2018")
print(df_old.shape)
print(df_old.columns)

Data - 2018 to 2019
(196894, 67)
Index([u':@computed_region_26cr_cadq', u':@computed_region_2dwj_jsy4',
       u':@computed_region_6qbp_sg9q', u':@computed_region_ajp5_b2md',
       u':@computed_region_h4ep_8xdi', u':@computed_region_nqbw_i6c3',
       u':@computed_region_qgnn_b9vv', u':@computed_region_y6ts_4iup',
       u'Analysis Neighborhood', u'Analysis Neighborhoods', u'CAD Number',
       u'CNN', u'Central Market/Tenderloin Boundary Polygon - Updated',
       u'Current Police Districts', u'Current Supervisor Districts',
       u'Filed Online', u'HSOC Zones as of 2018-06-05', u'Incident Category',
       u'Incident Code', u'Incident Date', u'Incident Datetime',
       u'Incident Day of Week', u'Incident Description', u'Incident ID',
       u'Incident Number', u'Incident Subcategory', u'Incident Time',
       u'Incident Year', u'Intersection', u'Latitude', u'Longitude',
       u'OWED Public Spaces', u'Parks Alliance CPSI (27+TL sites)',
       u'Police District', u'Report Datetime

In [10]:
## Data cleaning - keep variables in df_new

df_new = df_new[['incident_category', 'incident_date', 
                 'incident_day_of_week', 'incident_description', 'incident_number',
                 'incident_time', 'latitude', 'longitude', 'police_district']]

df_new = df_new.rename(columns={'incident_day_of_week': 'incident_dow'})


In [11]:
print(df_old.columns)

Index([u':@computed_region_2dwj_jsy4', u':@computed_region_4isq_27mq',
       u':@computed_region_6ezc_tdp2', u':@computed_region_6pnf_4xz7',
       u':@computed_region_9dfj_4gjx', u':@computed_region_9jxd_iqea',
       u':@computed_region_bh8s_q3mv', u':@computed_region_fcz8_est8',
       u':@computed_region_fyvs_ahh9', u':@computed_region_h4ep_8xdi',
       u':@computed_region_n4xg_c4py', u':@computed_region_nqbw_i6c3',
       u':@computed_region_p5aj_wyqh', u':@computed_region_pigm_ib2e',
       u':@computed_region_rxqg_mtj9', u':@computed_region_yftq_j783',
       u'Address', u'Analysis Neighborhoods', u'Category',
       u'Current Police Districts', u'Current Supervisor Districts', u'Date',
       u'DayOfWeek', u'Descript', u'IncidntNum', u'Location',
       u'Location (address)', u'Location (city)', u'Location (state)',
       u'Location (zip)', u'PdDistrict', u'PdId', u'Resolution',
       u'SF Find Neighborhoods', u'Time', u'X', u'Y', u'address', u'category',
       u'date', u'

In [12]:
## Data cleaning - keep variables in df_old

df_old = df_old[['category', 'date', 'dayofweek', 'descript', 'incidntnum', 'time', 'x', 'y', 'pddistrict']]

df_old = df_old.rename(columns={
        'category': 'incident_category',
        'date': 'incident_date',
        'dayofweek': 'incident_dow',
        'descript':'incident_description',
        'incidntnum': 'incident_number',
        'time': 'incident_time',
        'x': 'latitude',
        'y': 'longitude',
        'pddistrict': 'police_district'})



In [13]:
df_old.incident_date.head()

0    2015-01-15T00:00:00.000
1    2015-05-02T00:00:00.000
2    2018-03-20T00:00:00.000
3    2016-06-03T00:00:00.000
4    2015-02-03T00:00:00.000
Name: incident_date, dtype: object

In [14]:
## Format Dates, dealing with different formats
# 1) create new column temp_date and store there datetimes recognized using first format
df_new['temp_date'] = pd.to_datetime(df_new['incident_date'], format='%Y/%m/%d', errors='coerce')
# 2) get not recognized rows
not_recongized = df_new[df_new.temp_date.isnull()]
# 3) inplace fill not recognized rows with values recognized using second format
df_new['temp_date'].fillna(pd.to_datetime(not_recongized['incident_date'], format='%Y/%m/%d', errors='coerce'), inplace=True)

# 4) delete starttime column
del df_new['incident_date']
# 5) inplace rename temp_starttime to starttime
df_new.rename(columns={"temp_date": "incident_date"}, inplace=True)

df_new['incident_date'] = pd.to_datetime(df_new['incident_date'],format='%m/%d/%Y') 

df_old['incident_date'] = df_old.incident_date.str.slice(0, 10)
df_old['incident_date'] = pd.to_datetime(df_old['incident_date'],format='%Y/%m/%d') 


In [15]:
## Merge incident data
print(len(df_old))
print(len(df_new))
df_incident = pd.concat([df_new, df_old])

print(df_incident.shape)
print(df_incident.head(5))


508850
196894
(705744, 9)
     incident_category incident_date               incident_description  \
0  Other Miscellaneous    2018-12-02      Burglary Tools, Possession Of   
1        Lost Property    2018-12-01                      Lost Property   
2              Assault    2019-03-18                            Battery   
3        Larceny Theft    2019-03-20  Theft, From Locked Vehicle, >$950   
4        Larceny Theft    2019-03-12    Theft, Other Property, $50-$200   

  incident_dow incident_number incident_time            latitude  \
0       Sunday       180908554         00:45  37.784908299430455   
1     Saturday       180908112         20:30   37.78640961281089   
2       Monday       190194129         14:01   37.75683373380551   
3    Wednesday       190199583         08:00  37.784006612420036   
4      Tuesday       196055103         13:30                 NaN   

             longitude police_district  
0  -122.40479506275997        Southern  
1  -122.40803623744476         C

In [16]:
# Create new meta-crime categories

df_incident['incident_category'] = df_incident['incident_category'].str.lower()

viol = ['assault','homicide','rape','forcible']
prop = ['larceny theft','larceny/theft','motor vehicle theft','motor vehicle theft','motor vehicle theft?'
        'vehicle theft','burglary', 'robbery','stolen property', 'arson']
df_incident['incident_type'] = 'other'
 
for i in viol: 
    df_incident['incident_type'] = np.where(df_incident['incident_category'] == i, 'violent', df_incident['incident_type'])
    
for i in prop: 
    df_incident['incident_type'] = np.where(df_incident['incident_category'] == i, 'property', df_incident['incident_type'])
    
df_incident['incident_type'].describe()

df_incident = df_incident.drop(['incident_category','incident_description', 'police_district'], axis=1)


In [17]:
## Drop if missing
print(df_incident.isnull().sum())
df_incident = df_incident.dropna()


incident_date          0
incident_dow           0
incident_number        0
incident_time          0
latitude           10378
longitude          10378
incident_type          0
dtype: int64


In [18]:
feather.write_dataframe(df_incident, 'data/sf_incidents.feather')
